In [8]:
from bs4 import BeautifulSoup
import requests
import time
import random
import os
from tqdm import tqdm

In [65]:
class PageParser():
    
    def __init__(self):
        self.url = 'https://www.avito.ru/rossiya/bilety_i_puteshestviya?p='
        self.main = 'https://www.avito.ru'
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) ' + 
                                  'AppleWebKit/537.73.11 (KHTML, like Gecko) Version/7.0.1 Safari/537.73.11'}
        self.results_counter = 10100
        self.links = list()
    
    def collect_links(self):
        '''
        Collect items links.
        return: str: item metadata
        '''
        i = 1
        while len(self.links) < self.results_counter:
            try:
                req = requests.get(self.url+str(i), headers=self.headers)
                html = req.text
                soup = BeautifulSoup(html, "lxml")
                h3 = soup.find_all('h3')
                for h in h3:
                    self.links.append(h.find('a')['href'])
                i += 1
                time.sleep(random.randint(1,4))
            except:
                return            
        return
    
    def parse_page(self, soup, url):
        '''
        Parse a page of an item and collect metadata.
        soup: instance of BeautifulSoup class: item page structure
        url: str: url of an item
        return:
        '''
        text = url
        info = list()
        
        info.append(soup.find('title'))
        info.append(soup.find('div', attrs={'class':'title-info-metadata-item'}))
        info.append(soup.find('div', attrs={'class':'item-description-text'}))
        info.append(soup.find('span', attrs={'class':'js-item-price'}))
        
        location = soup.find('div', attrs={'class':'item-map-location'})
        
        info.append(location.find('span', attrs={'itemprop': 'name'}))
        info.append(location.find(lambda tag: len(tag.attrs) == 0))
        
        for elem in info:
            if elem is not None:
                text += '\n'
                text += elem.text.strip(' \n')        
        return text
    
    def write_doc(self, text, url):
        '''
        Write down item's metadata as .txt file.
        text: str: metadata to be wtitten
        url: str: url of an item
        return: 
        '''
        if not os.path.exists('./avito_texts'):
            os.makedirs('./avito_texts')
        with open(r'./avito_texts/www.avito.ru%s.txt' %(url.replace('/', '=')), 'w', encoding='utf-8') as f:
            f.write(text)
        return
        
    def crowl(self):
        '''
        Iterate through all the items' pages, collect metadata and write it down.
        return:
        '''
        for link in tqdm(self.links):
            req = requests.get(self.main+link, headers=self.headers)
            html = req.text
            soup = BeautifulSoup(html, "lxml")
            text = self.parse_page(soup, req.url)
            self.write_doc(text, link)
            time.sleep(random.randint(1,4))
        return            
            

In [66]:
parser = PageParser()

In [ ]:
parser.collect_links()

In [ ]:
len(parser.links)

In [ ]:
parser.crowl()